In [29]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [30]:
SEASONS = list(range(1997, 2023))

In [31]:
P_DATA_DIR = "Playoff_DATA"
P_STANDINGS_DIR = os.path.join(P_DATA_DIR, "p_standings")
#PLAYOFFS_DIR = os.path.join(DATA_DIR, "playoffs")
P_SCORES_DIR = os.path.join(P_DATA_DIR, "p_scores")

In [32]:
#get html function
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)

        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout on {url}")
            continue
        else:
            break
    return html
    

In [36]:
# this function will scrape the playoff standings for each season
async def p_scrape_season(season):
    url = f"https://www.basketball-reference.com/playoffs/NBA_{season}_games.html"
    html = await get_html(url, "#info .button2.next")

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    p_standings_pages = [f"https://www.basketball-reference.com{l}" for l in href ]

    for url in p_standings_pages:
        save_path = os.path.join(P_STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [37]:
# this function will scrape the playoff scores for each season
for season in SEASONS:
    await p_scrape_season(season)

1997 NBA Playoffs Schedule and Results | Basketball-Reference.com
1998 NBA Playoffs Schedule and Results | Basketball-Reference.com
1999 NBA Playoffs Schedule and Results | Basketball-Reference.com
2000 NBA Playoffs Schedule and Results | Basketball-Reference.com


CancelledError: 